In [716]:
import pandas, numpy, requests
import warnings
warnings.filterwarnings("ignore")

### Access token - lasts 1 hour

In [773]:
# Access token required to make any requests
url = "https://auth.emsicloud.com/connect/token"
payload = "client_id=air&client_secret=e9bpmNV3&grant_type=client_credentials&scope=curricular_skills_prod:consumer"
token_header = {'content-type': 'application/x-www-form-urlencoded'}
token_response = requests.request("POST", url, data=payload, headers=token_header)
token = token_response.json()['access_token']
# Authorization header - token only good for 1 hour after retrieval
header = {'Authorization': 'Bearer '+token}

In [113]:
# Check Health of Service
url = "https://emsiservices.com/curricular-skills/status/"
health = requests.request("GET", url, headers=header)
print(health.text)

{"data":{"attributes":{"message":"Service is healthy.","healthy":true}}}


### Retrieve Courses Directly

In [687]:
def pull_courses(token, offset=0):
    search_url = "https://emsiservices.com/curricular-skills/courses/search"
    headers = {'Authorization': 'Bearer '+token,'Content-Type': "application/json"}
    # Required Params: site='ccc', isPublished=true, limit <= 100
    search_payload = '{ \"data\": { \"type\": \"courseSearch\", \"attributes\": { \"filter\": { \"site\": { \"in\": [\"ccc\"]}, \"isPublished\": true}, \"limit\": 100, \"offset\": '+str(offset)+' }}}'
    search = requests.post(url=search_url, data=search_payload, headers=headers)
    search = search.json()
    r = search['data']
    courses = []
    for course in r:
        cdict = {'id':course['id'], 'course_id':course['attributes']['courseId'], 'title':\
                course['attributes']['title'], 'description':course['attributes']['description'], \
                'skills':course['attributes']['skills'], 'credits':course['attributes']['credits'], \
                'url':course['attributes']['url'],'updated':course['attributes']['updatedAt']}
        courses.append(cdict)
    df = pandas.DataFrame(courses)
    return(df)

In [670]:
def pull_all_courses(token):
    offset = 0
    courses = []
    first_batch = pull_courses(token, offset=offset)
    courses.append(first_batch)
    total = search['meta']['totalAvailable']
    offset += 100
    while offset < total:
        batch = pull_courses(token, offset=offset)
        courses.append(batch)
        offset += 100
    df = pandas.concat(courses)
    df = df.reset_index(drop=True)
    return(df)

In [785]:
df = pull_all_courses(token)
df = df.rename(columns={'id':'course_id','course_id':'course_code'})
df.to_excel('C:/WDEMP/EMSI/Curricular Skills API/CCC_courses.xlsx')